In [194]:
import requests
from bs4 import BeautifulSoup
from time import sleep

In [3]:
url = 'https://www.latamairlines.com/co/es/ofertas-vuelos?dataFlight=%7B%22tripTypeSelected%22%3A%7B%22label%22%3A%22Ida%22%2C%22value%22%3A%22OW%22%7D%2C%22cabinSelected%22%3A%7B%22label%22%3A%22Economy%22%2C%22value%22%3A%22Economy%22%7D%2C%22passengerSelected%22%3A%7B%22adultQuantity%22%3A1%2C%22childrenQuantity%22%3A0%2C%22infantQuantity%22%3A0%7D%2C%22originSelected%22%3A%7B%22id%22%3A%22MDE_CO_AIRPORT%22%2C%22name%22%3A%22Jose+Maria+C%C3%B3rdova+Int.%22%2C%22city%22%3A%22Medell%C3%ADn%22%2C%22country%22%3A%22Colombia%22%2C%22iata%22%3A%22MDE%22%2C%22latitude%22%3A6.16454%2C%22longitude%22%3A-75.4231%2C%22timezone%22%3A-5%2C%22tz%22%3A%22America%2FBogota%22%2C%22type%22%3A%22AIRPORT%22%2C%22countryAlpha2%22%3A%22CO%22%2C%22airportIataCode%22%3A%22MDE%22%7D%2C%22destinationSelected%22%3A%7B%22id%22%3A%22LIM_PE_AIRPORT%22%2C%22name%22%3A%22Jorge+Ch%C3%A1vez+Int.%22%2C%22city%22%3A%22Lima%22%2C%22country%22%3A%22Per%C3%BA%22%2C%22iata%22%3A%22LIM%22%2C%22latitude%22%3A-12.0219%2C%22longitude%22%3A-77.114305%2C%22timezone%22%3A-5%2C%22tz%22%3A%22America%2FLima%22%2C%22type%22%3A%22AIRPORT%22%2C%22countryAlpha2%22%3A%22PE%22%2C%22airportIataCode%22%3A%22LIM%22%7D%2C%22dateGoSelected%22%3A%222021-01-13T17%3A00%3A00.000Z%22%2C%22dateReturnSelected%22%3Anull%2C%22redemption%22%3Afalse%7D'

In [6]:
from selenium import webdriver

## Para Firefox son estas configuraciones

Paso 1: Instanciar un <b>Driver</b> del navegador

In [83]:
options = webdriver.FirefoxOptions()
options.add_argument('-private')
driver = webdriver.Firefox(executable_path="../geckodriver", options=options)

Paso 2: Hacer que el navegador cargue la página web

In [84]:
driver.get(url)

Paso 3: Extraer la información de la Pagina

In [171]:
vuelos = driver.find_elements_by_xpath('//li[@class = "sc-fHSTwm danPdA"]')
len(vuelos)

10

In [172]:
vuelo_1=vuelos[0]
hora = vuelo_1.find_elements_by_xpath('.//span[@class="sc-lnmtFM ewiHQE"]')
duracion = vuelo_1.find_element_by_xpath('.//span[@class="sc-FQuPU dkpBdw"]').text 
len(hora),hora[0].text, hora[1].text, duracion

(2, '7:58 a. m.', '9:30 p. m.', '13 h 32 min')

Para acceder a elementos

In [173]:
boton_escalas = vuelo_1.find_element_by_xpath('.//div[@class="sc-iuDHTM gHiWYR"]/div')
boton_escalas.click()

In [175]:
escalas = vuelo_1.find_elements_by_xpath('//section[@data-test="section-info-leg"]')
cantidad_escalas = len(escalas)-1
cantidad_escalas

1

In [177]:
airp_salida = escalas[0].find_element_by_xpath('//div[@class="iataCode"]/span').text
hora_salida = escalas[0].find_element_by_xpath('//div[@class="iataCode"]/span[@class="time"]').text
airp_salida, hora_salida

('MDE', '7:58 a. m.')

Cerrar la ventana emergente:

In [178]:
boton_cerrar = driver.find_element_by_xpath('//div[@class="MuiPaper-root MuiDialog-paper MuiDialog-paperScrollPaper MuiDialog-paperWidthFalse col-sm-4 col-md-5 col-lg-5 MuiPaper-elevation24 MuiPaper-rounded"]//button')
boton_cerrar.click()

Abrir info de precios vuelo 1:

In [184]:
vuelo_2=vuelos[2]
vuelo_2.click()

In [17]:
tipo_tiquete = vuelo_1.find_elements_by_xpath('.//span [@class="sc-cFlXAS lhMviV"]')
tipo_tiquete = [tipo.text for tipo in tipo_tiquete]
tipo_tiquete

['basic', 'light', 'plus', 'top']

In [18]:
precios = vuelo_1.find_elements_by_xpath('.//span [@class="sc-dRCTWM frWsLu displayAmount"]')
precios = [precio.text for precio in precios]
precios

['$ 629.570,00', '$ 739.190,00', '$ 896.380,00', '$ 1.112.050,00']

# Funciones Automatizadas

In [215]:
def obtener_precios(vuelo):
    precio=[]
    #Se abre el menu
    vuelo.click()
    
    #sleep(1)
    delay=5
    try:
        element = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, './/span [@class="sc-cFlXAS lhMviV"]')))
        tipo_tiquete = vuelo.find_elements_by_xpath('.//span [@class="sc-cFlXAS lhMviV"]')
        tipo_tiquete = [tipo.text for tipo in tipo_tiquete]
        
        precios = vuelo.find_elements_by_xpath('.//span [@class="sc-dRCTWM frWsLu displayAmount"]')
        precios = [precio.text for precio in precios]
    
        [precio.append({tipo_tiquete[i]:precios[i]}) for i in range(len(precios))]
    except TimeoutException:
        print("La pagina tardó demasiado en cargar")
    
    #tipo_tiquete = vuelo.find_elements_by_xpath('.//span [@class="sc-cFlXAS lhMviV"]')
    #tipo_tiquete = [tipo.text for tipo in tipo_tiquete]
    
    
    #Se vuelve a cerrar el menu
    #vuelo.click()
    
    return precio

In [205]:
def obtener_escalas(vuelo_n):
    from time import sleep
    
    #Abrir el boton
    boton_escalas = vuelo_n.find_element_by_xpath('.//div[@class="sc-iuDHTM gHiWYR"]/div')
    boton_escalas.click()
    
    #Necesita un tiempo para cargar y luego buscar
    sleep(2)
    
    #Hallar cada escala
        
    escalas = vuelo_n.find_elements_by_xpath('//section[@data-test="section-info-leg"]')
    cantidad_escalas = len(escalas)-1
    
    info_escalas=[]
    
    #Por cada Escala ponemos la hora de salida
    for escala in escalas:
        #Aeropuerto de salida
        airp_salida = escala.find_element_by_xpath('.//div[@class="iataCode"]/span').text
        #hora de salida
        hora_salida = escala.find_element_by_xpath('.//div[@class="iataCode"]/span[@class="time"]').text
        #Aeropuerto de llegada
        airp_llegada = escala.find_element_by_xpath('.//div[@class="sc-epGmkI hynWkG"]//div[@class="iataCode"]/span').text
        #Hora de llegada
        hora_llegada = escala.find_element_by_xpath('.//div[@class="sc-epGmkI hynWkG"]//div[@class="iataCode"]/span[@class="time"]').text
        
        #Se añade la info de cada vuelo en la trayectoria
        data = {'origin': airp_salida,
                'dep_time': hora_salida, 
                'destiny': airp_llegada, 
                'arr_time': hora_llegada}
        info_escalas.append(data)
    
    #Para cerrar el botón
    boton_cerrar = driver.find_element_by_xpath('//div[@class="MuiPaper-root MuiDialog-paper MuiDialog-paperScrollPaper MuiDialog-paperWidthFalse col-sm-4 col-md-5 col-lg-5 MuiPaper-elevation24 MuiPaper-rounded"]//button')
    boton_cerrar.click()

    return info_escalas

In [145]:
def obtener_tiempo(vuelo):
    
    hora = vuelo.find_elements_by_xpath('.//span[@class="sc-lnmtFM ewiHQE"]')
    duracion = vuelo.find_element_by_xpath('.//span[@class="sc-FQuPU dkpBdw"]').text 
    
    salida = hora[0].text
    llegada = hora[1].text

    data = {'departure': salida,
            'arrival': llegada,
            'duration': duracion}
    
    
    return data

In [202]:
def obtener_info(driver):
    vuelos = driver.find_elements_by_xpath('//li[@class = "sc-fHSTwm danPdA"]')
    print(f'Se encontraron {len(vuelos)} vuelos.')
    print("Iniciando scraping...")
    info = []
    for vuelo in vuelos:
        #obtenemos los tiempos de cada vuelo
        tiempos = obtener_tiempo(vuelo)
        #Obtenemos los precios
        precios = obtener_precios(vuelo)
        #Obtenemos las escalas
        escalas = obtener_escalas(vuelo)
        
        info.append({'precios':precios,
                     'tiempos':tiempos,
                     'escalas':escalas})
    return info

## Técnica para crear delays dinámicos con Selenium

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [ ]:
#Se asigna un delay maximo
delay=5
try:
#Se crea un elemento con la espera, dirigido hacia el path en Xpath del objeto que queremos buscar cuando cargue
    element = WebDriverWait(path_anterior, delay).until(EC.presence_of_element_located((By.XPATH, '//li[@class="sc-fHSTwm danPdA"]')))
#Se continua el resto del procedimiento...
    info_vuelos = obtener_info(driver)
except TimeoutException:
    print("La pagina tardó demasiado en cargar")

# Continuamos el codigo

In [217]:
options = webdriver.FirefoxOptions()
options.add_argument('-private')
driver = webdriver.Firefox(executable_path="../geckodriver", options=options)
driver.get(url)
#sleep(1)
#Introducir una demora para cargar la pagina
delay=5
try:
    element = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//li[@class="sc-fHSTwm danPdA"]')))
    print("La pagina terminó de cargar")
    info_vuelos = obtener_info(driver)
except TimeoutException:
    print("La pagina tardó demasiado en cargar")
    info_vuelos = []
    
driver.close()

La pagina terminó de cargar
Se encontraron 10 vuelos.
Iniciando scraping...


In [219]:
info_vuelos

[{'precios': [{'basic': '$ 629.570,00'},
   {'light': '$ 739.190,00'},
   {'plus': '$ 896.380,00'},
   {'top': '$ 1.112.050,00'}],
  'tiempos': {'departure': '7:58 a. m.',
   'arrival': '9:30 p. m.',
   'duration': '13 h 32 min'},
  'escalas': [{'origin': 'MDE',
    'dep_time': '7:58 a. m.',
    'destiny': 'BOG',
    'arr_time': '8:50 a. m.'},
   {'origin': 'BOG',
    'dep_time': '6:27 p. m.',
    'destiny': 'LIM',
    'arr_time': '9:30 p. m.'}]},
 {'precios': [{'basic': '$ 629.570,00'},
   {'light': '$ 739.190,00'},
   {'plus': '$ 896.380,00'},
   {'top': '$ 1.112.050,00'}],
  'tiempos': {'departure': '8:29 a. m.',
   'arrival': '9:30 p. m.',
   'duration': '13 h 1 min'},
  'escalas': [{'origin': 'MDE',
    'dep_time': '8:29 a. m.',
    'destiny': 'BOG',
    'arr_time': '9:21 a. m.'},
   {'origin': 'BOG',
    'dep_time': '6:27 p. m.',
    'destiny': 'LIM',
    'arr_time': '9:30 p. m.'}]},
 {'precios': [{'basic': '$ 629.570,00'},
   {'light': '$ 739.190,00'},
   {'plus': '$ 896.380,00'